## Setting the Stage

In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L20DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

## Step 1

In [2]:
#Read candidates.txt and contributors.txt and insert their values into the respective tables.
with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)
        
with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)
        
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,first_name,id,last_name,middle_init,party
0,Mike,16,Huckabee,,R
1,Barack,20,Obama,,D
2,Rudolph,22,Giuliani,,R
3,Mike,24,Gravel,,D
4,John,26,Edwards,,D
5,Bill,29,Richardson,,D
6,Duncan,30,Hunter,,R
7,Dennis,31,Kucinich,,D
8,Ron,32,Paul,,R
9,Joseph,33,Biden,,D


## Step 2: Various Queries

In [3]:
query = '''SELECT * FROM candidates WHERE middle_init <> ""'''
viz_tables(candidate_cols, query)
print("{} candidates have a middle initial.".format(viz_tables(candidate_cols, query).shape[0]))

3 candidates have a middle initial.


### Do the following queries:
* Display the contributors where the state is "PA"
* Display the contributors where the amount contributed is greater than $\$1000.00$.
* Display the contributors from "UT" where the amount contributed is greater than $\$1000.00$.
* Display the contributors who didn't list their state
  - **Hint**:  Match `state` to the empty string
* Display the contributors from "WA" and "PA"
  - **Hint**:  You will need to use `IN ("WA", "PA")` in your `SELECT` statement.
* Display the contributors who contributed between $\$100.00$ and $\$200.00$.
  - **Hint**: You can use the `BETWEEN 100.00 and 200.00` clause.

#### Display the contributors where the state is "PA"

In [4]:
#display from the contributors table
contributors_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT * FROM contributors WHERE state = "PA"'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,-100.0,22,BETHLEHEM,2008-03-05,WALTER,71,BUCKLEY,W.,PA,1635 COUNTRY ROAD,,180155718
1,-100.0,22,BETHLEHEM,2008-03-05,MARJORIE,72,BUCKLEY,B.,PA,1635 COUNTRY ROAD,,180155718
2,-1046.0,32,Pittsburgh,2008-04-21,Philip,94,Raught,M,PA,4714 Plum Way,,15201
3,-1100.0,32,Yardley,2008-04-21,Judith,95,Ferrara,D,PA,1508 Waterford Road,,19067
4,200.0,37,BETHLEHEM,2008-01-22,JOHN,166,ABEL,H.,PA,422 THOMAS STREET,,180153316


#### Display the contributors where the amount contributed is greater than  $1000.00

In [5]:
query = '''SELECT * FROM contributors WHERE amount > 1000.0'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,1500.0,16,Monticello,2007-05-18,Mike,6,Akin,,AR,181 Baywood Lane,,71655
1,1300.0,16,North Augusta,2007-06-29,John D.,10,Allen,,SC,1052 Cannon Mill Drive,,29860
2,2300.0,16,Fort Smith,2007-06-21,R.D.,14,Altes,,AR,8600 Moody Road,,72903
3,2300.0,16,Hot Springs,2007-06-12,John,16,Anthony,,AR,211 Long Island Drive,,71913
4,2300.0,16,Conway,2007-04-11,David,22,Baker,,AR,2550 Adamsbrooke Drive,,72034
5,2300.0,20,Park City,2007-08-14,Linda,29,Buckel,,UT,PO Box 683130,,840683130
6,4600.0,20,Park City,2007-08-14,Linda,31,Buckel,,UT,PO Box 683130,,840683130
7,2300.0,20,Camden,2007-09-30,Blaine,34,Buck,M,ME,45 Eaton Ave,,48431752
8,1300.0,20,Boca Raton,2007-08-09,John,46,Buchanan,,FL,2025 NW 29th Rd,,334316303
9,2300.0,35,SOUTHPORT,2007-09-14,NIRA,136,ABRAMOWITZ,,CT,411 HARBOR ROAD,,68901376


#### Display the contributors from "UT" where the amount contributed is greater than  $1000.00

In [6]:
query = '''SELECT * FROM contributors WHERE amount > 1000.0 AND state = "UT"'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,2300.0,20,Park City,2007-08-14,Linda,29,Buckel,,UT,PO Box 683130,,840683130
1,4600.0,20,Park City,2007-08-14,Linda,31,Buckel,,UT,PO Box 683130,,840683130


#### Display the contributors who didn't list their state

In [7]:
query = '''SELECT * FROM contributors WHERE state = ""'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,-500.0,35,BRASSCHATT,2008-11-20,TRAVIS,126,BOURNE,,,LAGE KAART 77,,2930


#### Display the contributors from "WA" and "PA"

In [8]:
query = '''SELECT * FROM contributors WHERE state IN ("WA", "PA")'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,-700.0,22,SEATTLE,2008-03-05,SUZANNE,63,BURKE,M.,WA,3401 EVANSTON,,981038677
1,-100.0,22,BETHLEHEM,2008-03-05,WALTER,71,BUCKLEY,W.,PA,1635 COUNTRY ROAD,,180155718
2,-100.0,22,BETHLEHEM,2008-03-05,MARJORIE,72,BUCKLEY,B.,PA,1635 COUNTRY ROAD,,180155718
3,-1046.0,32,Pittsburgh,2008-04-21,Philip,94,Raught,M,PA,4714 Plum Way,,15201
4,-1100.0,32,Yardley,2008-04-21,Judith,95,Ferrara,D,PA,1508 Waterford Road,,19067
5,100.0,34,Mill Creek,2008-02-08,Rebecca,101,Aaronson,,WA,2000 Village Green Dr Apt 12,,980125787
6,100.0,34,Mill Creek,2008-02-14,Rebecca,107,Aaronson,,WA,2000 Village Green Dr Apt 12,,980125787
7,200.0,37,BETHLEHEM,2008-01-22,JOHN,166,ABEL,H.,PA,422 THOMAS STREET,,180153316


#### Display the contributors who contributed between  $100.00$ and  $200.00$

In [9]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 100.00 and 200.00'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,100.0,16,Pleasanton,2007-06-21,Don,4,Ahrens,,CA,4034 Rennellwood Way,,94566
1,100.0,16,Bentonville,2007-06-16,Charles,5,Akin,,AR,10187 Sugar Creek Road,,72712
2,200.0,16,Little Rock,2007-06-12,Rebecca,13,Allison,,AR,3206 Summit Court,,72227
3,100.0,16,Mount Sterling,2007-06-22,Robert,18,Arbogast,,OH,12900 State Route 56 SE,,43143
4,100.0,20,FAIRFAX,2007-09-19,Bruce,28,Buckheit,,VA,8904 KAREN DR,,220312731
5,100.0,20,Kansas City,2007-09-25,Thomas,32,Buck,,MO,4206 Terrace Street,,64111
6,200.0,20,Northfield,2007-09-12,Jay,33,Buck,K.,IL,1855 Old Willow Rd Unit 322,,600932918
7,100.0,20,Los Angeles,2007-07-10,Ida,38,Bucher,M,CA,1400 Warnall Ave,,900245333
8,200.0,20,Boca Raton,2007-08-14,John,47,Buchanan,,FL,2025 NW 29th Rd,,334316303
9,100.0,34,Mill Creek,2008-02-08,Rebecca,101,Aaronson,,WA,2000 Village Green Dr Apt 12,,980125787


## Step 3:  Sorting

In [10]:
query = '''SELECT * FROM candidates ORDER BY id DESC'''
viz_tables(candidate_cols, query)

,first_name,id,last_name,middle_init,party
0,Fred,41,Thompson,D.,R
1,Christopher,39,Dodd,J.,D
2,Tom,38,Tancredo,,R
3,John,37,McCain,,R
4,Samuel,36,Brownback,,R
5,Mitt,35,Romney,,R
6,Hillary,34,Clinton,R.,D
7,Joseph,33,Biden,,D
8,Ron,32,Paul,,R
9,Dennis,31,Kucinich,,D


### Do the following sorts on the `contributors` table:
* Sort the `candidates` table by `last_name`.
* Sort by the `amount` in decending order where `amount` is restricted to be between $\$1000.00$ and $\$5000.00$.
* Sort the contributors who donted between $\$1000.00$ and $\$5000.00$ by `candidate_id` and then by `amount` in descending order.
  - **Hint**:  Multiple orderings can be accomplished by separating requests after `ORDER BY` with commas.
  - e.g. `ORDER BY amount ASC, last_name DESC`

#### Sort the candidates table by last_name.

In [11]:
query = '''SELECT * FROM candidates ORDER BY last_name'''
viz_tables(candidate_cols, query)

,first_name,id,last_name,middle_init,party
0,Joseph,33,Biden,,D
1,Samuel,36,Brownback,,R
2,Hillary,34,Clinton,R.,D
3,Christopher,39,Dodd,J.,D
4,John,26,Edwards,,D
5,Rudolph,22,Giuliani,,R
6,Mike,24,Gravel,,D
7,Mike,16,Huckabee,,R
8,Duncan,30,Hunter,,R
9,Dennis,31,Kucinich,,D


#### Sort by the amount in decending order where amount is restricted to be between $1000.00$ and  $5000.00$.

In [13]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.0 and 5000.0 ORDER BY amount DESC'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,4600.0,20,Park City,2007-08-14,Linda,31,Buckel,,UT,PO Box 683130,,840683130
1,2600.0,37,MIAMI SPRINGS,2008-01-25,MARIA,160,ABATE,ELENA,FL,1291 NIGHTINGALE AVENUE,,331663832
2,2300.0,16,Fort Smith,2007-06-21,R.D.,14,Altes,,AR,8600 Moody Road,,72903
3,2300.0,16,Hot Springs,2007-06-12,John,16,Anthony,,AR,211 Long Island Drive,,71913
4,2300.0,16,Conway,2007-04-11,David,22,Baker,,AR,2550 Adamsbrooke Drive,,72034
5,2300.0,20,Park City,2007-08-14,Linda,29,Buckel,,UT,PO Box 683130,,840683130
6,2300.0,20,Camden,2007-09-30,Blaine,34,Buck,M,ME,45 Eaton Ave,,48431752
7,2300.0,35,SOUTHPORT,2007-09-14,NIRA,136,ABRAMOWITZ,,CT,411 HARBOR ROAD,,68901376
8,1500.0,16,Monticello,2007-05-18,Mike,6,Akin,,AR,181 Baywood Lane,,71655
9,1300.0,16,North Augusta,2007-06-29,John D.,10,Allen,,SC,1052 Cannon Mill Drive,,29860


#### Sort the contributors who donted between  $1000.00$ and  $5000.00$  by candidate_id and then by amount in descending order.

In [14]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.0 and 5000.0 ORDER BY candidate_id, amount DESC'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,id,last_name,middle_name,state,street_1,street_2,zip
0,2300.0,16,Fort Smith,2007-06-21,R.D.,14,Altes,,AR,8600 Moody Road,,72903
1,2300.0,16,Hot Springs,2007-06-12,John,16,Anthony,,AR,211 Long Island Drive,,71913
2,2300.0,16,Conway,2007-04-11,David,22,Baker,,AR,2550 Adamsbrooke Drive,,72034
3,1500.0,16,Monticello,2007-05-18,Mike,6,Akin,,AR,181 Baywood Lane,,71655
4,1300.0,16,North Augusta,2007-06-29,John D.,10,Allen,,SC,1052 Cannon Mill Drive,,29860
5,1000.0,16,North Augusta,2007-06-11,John D.,9,Allen,,SC,1052 Cannon Mill Drive,,29860
6,1000.0,16,Conway,2007-05-18,John W.,11,Allison,,AR,P.O. Box 1089,,72033
7,1000.0,16,Little Rock,2007-04-25,Rebecca,12,Allison,,AR,3206 Summit Court,,72227
8,1000.0,16,Pine Bluff,2007-05-18,Omar,20,Atiq,,AR,7200 S Hazel Street,,71603
9,1000.0,16,Pine Bluff,2007-06-27,Omar,21,Atiq,,AR,7200 S Hazel Street,,71603


## Step 4:  Selecting Columns

In [15]:
query = '''SELECT last_name, party FROM candidates'''
viz_tables(['last_name', 'party'], query)
query = '''SELECT DISTINCT party FROM candidates'''
viz_tables(['party'], query)

,party
0,R
1,D


#### Get the first and last name of contributors.  Make sure each row has distinct values.

In [16]:
query = '''SELECT DISTINCT first_name, last_name FROM contributors'''
viz_tables(['first_name', 'last_name'], query)

,first_name,last_name
0,Steven,Agee
1,Don,Ahrens
2,Charles,Akin
3,Mike,Akin
4,Rebecca,Akin
5,Brittni,Aldridge
6,John D.,Allen
7,John W.,Allison
8,Rebecca,Allison
9,R.D.,Altes


## Step 5:  Altering Tables

In [17]:
cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
viz_tables(candidate_cols, '''SELECT * FROM candidates''')
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")] # regenerate columns with full_name
query = '''SELECT id, last_name, first_name FROM candidates''' # Select a few columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)

update = '''UPDATE candidates SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,first_name,full_name,id,last_name,middle_init,party
0,Mike,"Huckabee, Mike",16,Huckabee,,R
1,Barack,"Obama, Barack",20,Obama,,D
2,Rudolph,"Giuliani, Rudolph",22,Giuliani,,R
3,Mike,"Gravel, Mike",24,Gravel,,D
4,John,"Edwards, John",26,Edwards,,D
5,Bill,"Richardson, Bill",29,Richardson,,D
6,Duncan,"Hunter, Duncan",30,Hunter,,R
7,Dennis,"Kucinich, Dennis",31,Kucinich,,D
8,Ron,"Paul, Ron",32,Paul,,R
9,Joseph,"Biden, Joseph",33,Biden,,D


In [18]:
update = '''UPDATE candidates SET full_name = "Eventual Winner" WHERE last_name = "Obama"'''
cursor.execute(update)
update = '''UPDATE candidates SET full_name = "Eventual Loser" WHERE last_name = "Romney"'''
cursor.execute(update)
viz_tables(candidate_cols, query)

,first_name,full_name,id,last_name,middle_init,party
0,Mike,"Huckabee, Mike",16,Huckabee,,R
1,Barack,Eventual Winner,20,Obama,,D
2,Rudolph,"Giuliani, Rudolph",22,Giuliani,,R
3,Mike,"Gravel, Mike",24,Gravel,,D
4,John,"Edwards, John",26,Edwards,,D
5,Bill,"Richardson, Bill",29,Richardson,,D
6,Duncan,"Hunter, Duncan",30,Hunter,,R
7,Dennis,"Kucinich, Dennis",31,Kucinich,,D
8,Ron,"Paul, Ron",32,Paul,,R
9,Joseph,"Biden, Joseph",33,Biden,,D


#### Add a new column to the contributors table called full_name. The value in that column should be in the form last_name, first_name.

In [48]:
cursor.execute('''ALTER TABLE contributors ADD COLUMN full_name TEXT''')
contributors_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]# regenerate columns with full_name
query = '''SELECT id, last_name, first_name FROM contributors''' # Select columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)
update = '''UPDATE contributors SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM contributors'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,full_name,id,last_name,middle_name,state,street_1,street_2,zip
0,500.0,16,Floyd,2007-06-30,Steven,"Agee, Steven",1,Agee,,VA,549 Laurel Branch Road,,24091
1,250.0,16,Pleasanton,2007-05-16,Don,"Ahrens, Don",2,Ahrens,,CA,4034 Rennellwood Way,,94566
2,50.0,16,Pleasanton,2007-06-18,Don,"Ahrens, Don",3,Ahrens,,CA,4034 Rennellwood Way,,94566
3,100.0,16,Pleasanton,2007-06-21,Don,"Ahrens, Don",4,Ahrens,,CA,4034 Rennellwood Way,,94566
4,100.0,16,Bentonville,2007-06-16,Charles,"Akin, Charles",5,Akin,,AR,10187 Sugar Creek Road,,72712
5,1500.0,16,Monticello,2007-05-18,Mike,"Akin, Mike",6,Akin,,AR,181 Baywood Lane,,71655
6,500.0,16,Monticello,2007-05-18,Rebecca,"Akin, Rebecca",7,Akin,,AR,181 Baywood Lane,,71655
7,250.0,16,Washington,2007-06-06,Brittni,"Aldridge, Brittni",8,Aldridge,,DC,"808 Capitol Square Place, SW",,20024
8,1000.0,16,North Augusta,2007-06-11,John D.,"Allen, John D.",9,Allen,,SC,1052 Cannon Mill Drive,,29860
9,1300.0,16,North Augusta,2007-06-29,John D.,"Allen, John D.",10,Allen,,SC,1052 Cannon Mill Drive,,29860


#### Change the value in the full_name column to the string "Too Much" if someone donated more than  $1000.00$

In [49]:
update = '''UPDATE contributors SET full_name = "Too Much" WHERE amount > 1000.0'''
cursor.execute(update)
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,full_name,id,last_name,middle_name,state,street_1,street_2,zip
0,500.0,16,Floyd,2007-06-30,Steven,"Agee, Steven",1,Agee,,VA,549 Laurel Branch Road,,24091
1,250.0,16,Pleasanton,2007-05-16,Don,"Ahrens, Don",2,Ahrens,,CA,4034 Rennellwood Way,,94566
2,50.0,16,Pleasanton,2007-06-18,Don,"Ahrens, Don",3,Ahrens,,CA,4034 Rennellwood Way,,94566
3,100.0,16,Pleasanton,2007-06-21,Don,"Ahrens, Don",4,Ahrens,,CA,4034 Rennellwood Way,,94566
4,100.0,16,Bentonville,2007-06-16,Charles,"Akin, Charles",5,Akin,,AR,10187 Sugar Creek Road,,72712
5,1500.0,16,Monticello,2007-05-18,Mike,Too Much,6,Akin,,AR,181 Baywood Lane,,71655
6,500.0,16,Monticello,2007-05-18,Rebecca,"Akin, Rebecca",7,Akin,,AR,181 Baywood Lane,,71655
7,250.0,16,Washington,2007-06-06,Brittni,"Aldridge, Brittni",8,Aldridge,,DC,"808 Capitol Square Place, SW",,20024
8,1000.0,16,North Augusta,2007-06-11,John D.,"Allen, John D.",9,Allen,,SC,1052 Cannon Mill Drive,,29860
9,1300.0,16,North Augusta,2007-06-29,John D.,Too Much,10,Allen,,SC,1052 Cannon Mill Drive,,29860


## Step 6: Aggregation

In [50]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")] # You've already done this part.  I just need to do it here b/c I haven't yet.
function = '''SELECT *, MAX(amount) AS max_amount FROM contributors'''
viz_tables(contributor_cols, function)

,amount,candidate_id,city,date,first_name,full_name,id,last_name,middle_name,state,street_1,street_2,zip
0,4600.0,20,Park City,2007-08-14,Linda,Too Much,31,Buckel,,UT,PO Box 683130,,840683130


#### Count how many donations there were above  $1000.00$

In [59]:
query = '''SELECT COUNT(id) FROM contributors WHERE amount > 1000.0'''
cursor.execute(query).fetchall()

[(12,)]

#### Calculate the average donation.

In [60]:
query = '''SELECT AVG(amount) FROM contributors'''
cursor.execute(query).fetchall()

[(3.418114285714276,)]

#### Calculate the average contribution from each state and display in a table.

In [62]:
function = '''SELECT state,AVG(amount) FROM contributors GROUP BY state'''
cols = ['state', 'avg']
viz_tables(cols, function)

,avg,state
0,-500.000000,
1,403.333333,AK
2,1183.333333,AR
3,120.000000,AZ
4,-217.988261,CA
5,-1455.750000,CO
6,2300.000000,CT
7,-309.982000,DC
8,-135.000000,FL
9,250.000000,IA


## Step 7: DELETE

In [69]:
deletion = '''DELETE FROM contributors WHERE last_name = "Ahrens"'''
cursor.execute(deletion)
query = '''SELECT * FROM contributors WHERE last_name = "Ahrens"'''
cursor.execute(query).fetchall()

[]

## Step 8: LIMIT

#### Query and display the ten most generous donors.

In [71]:
query = '''SELECT * FROM contributors ORDER BY amount DESC LIMIT 10'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,full_name,id,last_name,middle_name,state,street_1,street_2,zip
0,4600.0,20,Park City,2007-08-14,Linda,Too Much,31,Buckel,,UT,PO Box 683130,,840683130
1,2600.0,37,MIAMI SPRINGS,2008-01-25,MARIA,Too Much,160,ABATE,ELENA,FL,1291 NIGHTINGALE AVENUE,,331663832
2,2300.0,16,Fort Smith,2007-06-21,R.D.,Too Much,14,Altes,,AR,8600 Moody Road,,72903
3,2300.0,16,Hot Springs,2007-06-12,John,Too Much,16,Anthony,,AR,211 Long Island Drive,,71913
4,2300.0,16,Conway,2007-04-11,David,Too Much,22,Baker,,AR,2550 Adamsbrooke Drive,,72034
5,2300.0,20,Park City,2007-08-14,Linda,Too Much,29,Buckel,,UT,PO Box 683130,,840683130
6,2300.0,20,Camden,2007-09-30,Blaine,Too Much,34,Buck,M,ME,45 Eaton Ave,,48431752
7,2300.0,35,SOUTHPORT,2007-09-14,NIRA,Too Much,136,ABRAMOWITZ,,CT,411 HARBOR ROAD,,68901376
8,1500.0,16,Monticello,2007-05-18,Mike,Too Much,6,Akin,,AR,181 Baywood Lane,,71655
9,1300.0,16,North Augusta,2007-06-29,John D.,Too Much,10,Allen,,SC,1052 Cannon Mill Drive,,29860


#### Query and display the ten least generous donors who donated a positive amount of money.

In [72]:
query = '''SELECT * FROM contributors WHERE amount > 0.0 ORDER BY amount LIMIT 10'''
viz_tables(contributors_cols, query)

,amount,candidate_id,city,date,first_name,full_name,id,last_name,middle_name,state,street_1,street_2,zip
0,25.0,20,Dana Point,2007-08-16,Steve,"Buckler, Steve",27,Buckler,,CA,24351 Armada Dr,,926291306
1,25.0,20,La Verne,2007-07-26,Ryan,"Harrison, Ryan",50,Harrison,,CA,2247 3rd St,,917504918
2,25.0,34,West Yarmouth,2008-02-26,Elaine,"Aarons, Elaine",102,Aarons,,MA,481 Buck Island Rd Apt 17A,APT 17A,26733300
3,25.0,35,SALT LAKE CITY,2007-09-17,PATRICIA,"ABEGG, PATRICIA",141,ABEGG,T.,UT,1862 E. 5150 S.,,841176911
4,25.0,35,SALT LAKE CITY,2007-08-06,PATRICIA,"ABEGG, PATRICIA",144,ABEGG,T.,UT,1862 E. 5150 S.,,841176911
5,25.0,35,SALT LAKE CITY,2007-07-10,PATRICIA,"ABEGG, PATRICIA",145,ABEGG,T.,UT,1862 E. 5150 S.,,841176911
6,25.0,37,WATERTOWN,2008-01-09,PETER,"ABAIR, PETER",161,ABAIR,,MA,40 EVANS STREET,,24722150
7,35.0,37,WHITE LAKE,2008-01-07,PAULINE,"ABBO, PAULINE",159,ABBO,MORENCY,MI,10720 JACOB LANE,,483862274
8,50.0,16,Springfield,2007-06-28,William,"Ardle, William",19,Ardle,,OH,412 Dakota Avenue,,45504
9,50.0,20,Dana Point,2007-07-30,Steve,"Buckler, Steve",26,Buckler,,CA,24351 Armada Dr,,926291306


## Save

In [73]:
db.commit()

In [74]:
db.close()